# Manipulation of Quantum States in QuAIRKit

QuAIRKit uses the `State` class to represent quantum states. In this tutorial, we will learn how to manipulate quantum states in QuAIRKit.

**Table of Contents**

- [Creating of states](#Creation-of-states)
- [Information extraction](#Information-extraction)
- [Manipulation of states](#Manipulation-of-states)
- [Interaction with environments](#Interaction-with-environments)

In [1]:
import torch
import traceback
from quairkit import to_state
from quairkit.database import *

## Creation of states

In QuAIRKit, states can be generated in two ways. 

One common method is to call a state generation function form the QuAIRKit database.

In [2]:
num_qubits = 2

state = zero_state(num_qubits) # |00>
state = bell_state(num_qubits) # (|00> + |11>) / sqrt(2)
state = isotropic_state(num_qubits, prob=0.1) # isotropic state

state = random_state(num_qubits, rank=1) # random 2-qubit pure state
state = random_state(num_qubits, is_real=True) # random 2-qubit real state
state = random_state(num_qubits, size=1000) # 1000 random 2-qubit states

Or, one can use the function `to_state` to create a quantum state. The function `to_state` can convert a *torch.Tensor* or a *numpy.ndarray* instance to a State instance. The accepted shape of the input tensor is listed in the following table.

|                | single  state       | batch  states       |
|----------------|:---------------------:|:---------------------:|
| state vector   | [d], [1, d], [d, 1] | [d1, ..., dn, d, 1] |
| density matrix | [d, d]              | [d1, ..., dn, d, d] |


In [3]:
data = haar_state_vector(num_qubits) # random 3-qubit state vector
state = to_state(data)
print(state.backend, state.numel())

data = random_density_matrix(num_qubits) # random 3-qubit density matrix
state = to_state(data)
print(state.backend, state.numel())

state_vector 1
density_matrix 1


In [4]:
data = torch.stack([haar_state_vector(num_qubits) for _ in range(1000)]) # 1000 3-qubit state vector
state = to_state(data)
print(state.backend, state.numel())

data = torch.stack([random_density_matrix(num_qubits) for _ in range(1000)]) # 1000 3-qubit density matrix
state = to_state(data)
print(state.backend, state.numel())

state_vector 1000
density_matrix 1000


## Information extraction

A State object has several methods to retrieve its quantum information. 

We start with a batch of 3 random generated states $\{ |\psi_j\rangle \}_{j=1}^3$.

In [5]:
state = random_state(num_qubits, rank=1, size=3) # 3 random 2-qubit pure states
print(state)


---------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[ 0.3 +0.33j  0.61-0.49j -0.32+0.15j  0.18+0.19j]
 # 1:
[-0.29+0.37j -0.6 -0.15j  0.02+0.52j -0.29-0.19j]
 # 2:
[-0.44+0.1j  -0.14-0.14j -0.51+0.44j  0.23+0.51j]
---------------------------------------------------



One can retrieve the ket $\{ |\psi_j\rangle \}_{j=1}^3$, bra $\{ \langle\psi_j| \}_{j=1}^3$ and density matrix $\{ |\psi_j\rangle\langle\psi_j| \}_{j=1}^3$ of these states. Note that ket and bra properties are only available for pure states.

In [6]:
print(state.ket.shape)
print(state.bra.shape)
print(state.density_matrix.shape)

torch.Size([3, 4, 1])
torch.Size([3, 1, 4])
torch.Size([3, 4, 4])


Useful information can be retrieved by the functions of State instances.

In [7]:
print("The trace of these states are", state.trace())
print("The rank of these states are", state.rank)
print("The size of these states are", state.dim)
print("The shape of vectorization of these states are", state.vec.shape)

The trace of these states are tensor([1.0000+0.j, 1.0000+0.j, 1.0000+0.j])
The rank of these states are 1
The size of these states are 4
The shape of vectorization of these states are torch.Size([3, 16, 1])


In [8]:
print("The number of systems in these states are", state.num_systems)
print("Are these states qubits?", state.are_qubits())
print("Are these states qutrits?", state.are_qutrits())

The number of systems in these states are 2
Are these states qubits? True
Are these states qutrits? False


## Manipulation of states

State instances can easily permute subsystems by changing the value of `system_seq`. However, such behavior does not affect its output on ket, bra and density matrix, in which case `system_seq` would be reset to default sequence.

In [9]:
state.system_seq = [1, 0]
print(state)


---------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [1, 0]
 Batch size: [3]

 # 0:
[ 0.3 +0.33j -0.32+0.15j  0.61-0.49j  0.18+0.19j]
 # 1:
[-0.29+0.37j  0.02+0.52j -0.6 -0.15j -0.29-0.19j]
 # 2:
[-0.44+0.1j  -0.51+0.44j -0.14-0.14j  0.23+0.51j]
---------------------------------------------------



Clone a state, or change dtype, device

In [10]:
print("The dtype of these states are", state.dtype)
print("The device of these states are", state.device)

new_state = state.clone().to(dtype=torch.complex128, device="cpu") # change to "cuda" if gpu is available
print("The dtype of new states are", new_state.dtype)
print("The device of new states are", new_state.device)

The dtype of these states are torch.complex64
The device of these states are cpu
The dtype of new states are torch.complex128
The device of new states are cpu


State instances support direct indexing, for example retrieve the second and third state in the batch.

In [11]:
print(state[1:])


---------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [2]

 # 0:
[-0.29+0.37j -0.6 -0.15j  0.02+0.52j -0.29-0.19j]
 # 1:
[-0.44+0.1j  -0.14-0.14j -0.51+0.44j  0.23+0.51j]
---------------------------------------------------



Or if the length of batch dimension is larger than 1, one can index elements in a batch dimension. See [torch.index_select](https://pytorch.org/docs/stable/generated/torch.index_select.html) for more understanding.

In [12]:
print(state.index_select(0, torch.tensor([1, 2])))


---------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [2]

 # 0:
[-0.29+0.37j -0.6 -0.15j  0.02+0.52j -0.29-0.19j]
 # 1:
[-0.44+0.1j  -0.14-0.14j -0.51+0.44j  0.23+0.51j]
---------------------------------------------------



## Interaction with environments

State instances can be sent to a quantum environment for further processing.

In [13]:
unitary = random_unitary(num_qubits=1)
kraus = random_channel(num_qubits=1)
choi = random_channel(num_qubits=1, target='choi')

In [14]:
state = state.evolve(unitary, sys_idx=[1])
print(state)


---------------------------------------------------
 Backend: state_vector
 System dimension: [2, 2]
 System sequence: [1, 0]
 Batch size: [3]

 # 0:
[ 0.73-0.34j  0.04+0.32j -0.3 +0.27j -0.28-0.11j]
 # 1:
[-0.53+0.17j -0.08+0.11j -0.13+0.53j -0.07+0.61j]
 # 2:
[-0.31+0.07j  0.07+0.78j -0.36+0.09j -0.38-0.08j]
---------------------------------------------------



Note that when pure states are sent to noisy environments, they will be automatically converted to mixed states, in which case their ket and bra properties will be lost.

In [15]:
state = state.transform(kraus, sys_idx=[0])
state = state.transform(choi, sys_idx=[1], repr_type='choi')
print(state)


---------------------------------------------------
 Backend: density_matrix
 System dimension: [2, 2]
 System sequence: [0, 1]
 Batch size: [3]

 # 0:
[[ 0.49+0.j   -0.1 +0.14j  0.32+0.24j -0.17+0.09j]
 [-0.1 -0.14j  0.07+0.j    0.  -0.15j  0.07+0.03j]
 [ 0.32-0.24j  0.  +0.15j  0.36-0.j   -0.06+0.16j]
 [-0.17-0.09j  0.07-0.03j -0.06-0.16j  0.08-0.j  ]]
 # 1:
[[ 0.67+0.j    0.22+0.01j -0.02-0.08j -0.02-0.06j]
 [ 0.22-0.01j  0.22+0.j   -0.08-0.1j   0.03-0.06j]
 [-0.02+0.08j -0.08+0.1j   0.08+0.j    0.01+0.04j]
 [-0.02+0.06j  0.03+0.06j  0.01-0.04j  0.03-0.j  ]]
 # 2:
[[ 0.49-0.j    0.11+0.17j -0.26-0.05j  0.09-0.19j]
 [ 0.11-0.17j  0.11+0.j   -0.02+0.1j  -0.06-0.07j]
 [-0.26+0.05j -0.02-0.1j   0.3 +0.j   -0.06+0.13j]
 [ 0.09+0.19j -0.06+0.07j -0.06-0.13j  0.1 +0.j  ]]
---------------------------------------------------



In [17]:
try:
    state.ket
except NotImplementedError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\Cloud\AppData\Local\Temp\ipykernel_8428\2901163604.py", line 4, in <module>
    state.ket
  File "c:\users\cloud\quair-platform\quairkit\core\state\backend\density_matrix.py", line 99, in ket
    raise NotImplementedError(
NotImplementedError: Mixed state does not support state vector representation.If you are looking for the vectorization of the mixed state, please call the 'vec' property.
